# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [31]:
# load csv file
file = pd.read_csv("../output_data/WeatherPy.csv")

# create a dataframe with the csv file
cities_df = pd.DataFrame(file)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mataura,-46.19,168.86,66.00,67,100,10.00,NZ,1608257543
1,Rikitea,-23.12,-134.97,77.43,75,2,17.29,PF,1608257544
2,Kapaa,22.08,-159.32,80.60,69,20,14.99,US,1608257545
3,Ushuaia,-54.80,-68.30,42.80,75,75,29.97,AR,1608257267
4,Punta Arenas,-53.15,-70.92,48.20,61,100,9.17,CL,1608257548


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [33]:
# create locations variable
location = cities_df[["Lat", "Lng"]]

# create humidity variable
humidity = cities_df["Humidity"]

In [34]:
# plot heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(location, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [38]:
# Narrow down cities to fit weather conditions
nice_low_temp = cities_df[cities_df["Max Temp"]>=70]
nice_high_temp = nice_low_temp[nice_low_temp["Max Temp"]<=95] 
nice_humidity = nice_high_temp[nice_high_temp["Humidity"]<=50] 
nice_cloudiness = nice_humidity[nice_humidity["Cloudiness"]<=40]
nice_weather = nice_cloudiness[nice_cloudiness["Wind Speed"]<=10]

nice_weather.dropna(how="any", inplace=True)

nice_weather

<ipython-input-38-dc7edca997b5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nice_weather.dropna(how="any", inplace=True)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,General Roca,-39.03,-67.58,75.20,29,0,6.31,AR,1608257567
24,Busselton,-33.65,115.33,73.99,48,0,3.00,AU,1608257572
57,Mount Isa,-20.73,139.50,95.00,39,40,8.05,AU,1608257616
128,Djibo,14.10,-1.63,75.09,20,7,8.28,BF,1608257699
213,San Juan,-31.54,-68.54,84.70,37,1,10.00,AR,1608257804
289,Emerald,-23.53,148.17,93.99,46,3,6.93,AU,1608257792
318,Yeppoon,-23.13,150.73,93.20,41,40,6.93,AU,1608257929
549,Juba,4.85,31.58,75.81,29,0,4.65,SS,1608258215


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
hotel_df = nice_weather

In [40]:
hotel_df["Hotel Name"] = ''

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng in dataframe to use as the locations
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and convert to json
    name_address = requests.get(base_url, params=params).json()

    # Use a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

<ipython-input-40-1b29676bc91f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ''
/Users/nickcresta/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
21,General Roca,-39.03,-67.58,75.20,29,0,6.31,AR,1608257567,Hotel El Recreo
24,Busselton,-33.65,115.33,73.99,48,0,3.00,AU,1608257572,Observatory Guest House
57,Mount Isa,-20.73,139.50,95.00,39,40,8.05,AU,1608257616,ibis Styles Mt Isa Verona
128,Djibo,14.10,-1.63,75.09,20,7,8.28,BF,1608257699,Centre Des Ressources DPENA
213,San Juan,-31.54,-68.54,84.70,37,1,10.00,AR,1608257804,Hotel Provincial
289,Emerald,-23.53,148.17,93.99,46,3,6.93,AU,1608257792,The Irish Village
318,Yeppoon,-23.13,150.73,93.20,41,40,6.93,AU,1608257929,Villa Mar Colina
549,Juba,4.85,31.58,75.81,29,0,4.65,SS,1608258215,ROYAL PALACE HOTEL - JUBA


In [41]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [42]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))